In [11]:
import napari
import numpy as np
from numpy.lib.format import open_memmap
import myfunctions as mf
import os

In [2]:
OS = 'Windows'
exp = mf.exp_list()[0]

hypervolume = open_memmap(os.path.join(mf.OS_path(exp, OS), 'hypervolume.npy'), mode='r')
hypervolume_mask = open_memmap(os.path.join(mf.OS_path(exp, OS), 'hypervolume_mask.npy'), mode='r')
print(hypervolume_mask.shape)
print(hypervolume.shape)

(55, 270, 500, 500)
(55, 270, 500, 500)


In [4]:
viewer = napari.Viewer()

images = [viewer.add_image(hypervolume, name='Volume', opacity=0.4)]
labels = [viewer.add_labels(hypervolume_mask, name='Labels', blending='additive', opacity=0.8)]

settings = napari.settings.get_settings()
settings.application.playback_fps = 5
viewer.dims.current_step = (0, 0)